In [10]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Initial employee data
initial_employees = {
    "Employee1": {"name": "Rohan", "age": 30, "salary": 55000.0},
    "Employee2": {"name": "Shivam", "age": 28, "salary": 48000.75},
    "Employee3": {"name": "Rahul", "age": 35, "salary": 65000.0},
    "Employee4": {"name": "Priya", "age": 27, "salary": 50000.5}
}

employees = pd.DataFrame(
    [{"ID": key, "Name": val["name"], "Age": val["age"], "Salary": val["salary"]}
     for key, val in initial_employees.items()]
)

emp_id_counter = len(initial_employees) + 1

# Setup main window
root = tk.Tk()
root.title("Employee Tracking System")
root.geometry("950x600")

# Frames for layout
frame_inputs = ttk.Frame(root, padding=10)
frame_inputs.pack(side=tk.TOP, fill=tk.X)

frame_buttons = ttk.Frame(root, padding=10)
frame_buttons.pack(side=tk.TOP, fill=tk.X)

frame_output = ttk.Frame(root, padding=10)
frame_output.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

frame_plot = ttk.Frame(root, padding=10)
frame_plot.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)

# Input fields
ttk.Label(frame_inputs, text="Name:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
entry_name = ttk.Entry(frame_inputs, width=20)
entry_name.grid(row=0, column=1, padx=5, pady=5)

ttk.Label(frame_inputs, text="Age:").grid(row=0, column=2, padx=5, pady=5, sticky=tk.W)
entry_age = ttk.Entry(frame_inputs, width=10)
entry_age.grid(row=0, column=3, padx=5, pady=5)

ttk.Label(frame_inputs, text="Salary:").grid(row=0, column=4, padx=5, pady=5, sticky=tk.W)
entry_salary = ttk.Entry(frame_inputs, width=15)
entry_salary.grid(row=0, column=5, padx=5, pady=5)

ttk.Label(frame_inputs, text="Search by ID or Name:").grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
entry_search = ttk.Entry(frame_inputs, width=20)
entry_search.grid(row=1, column=1, padx=5, pady=5)

# Treeview for displaying employees
tree = ttk.Treeview(frame_output, columns=("ID", "Name", "Age", "Salary"), show="headings", selectmode='browse')
tree.heading("ID", text="Employee ID")
tree.heading("Name", text="Name")
tree.heading("Age", text="Age")
tree.heading("Salary", text="Salary")
tree.pack(fill=tk.BOTH, expand=True)

def refresh_tree():
    for row in tree.get_children():
        tree.delete(row)
    for _, row in employees.iterrows():
        tree.insert("", tk.END, values=(row["ID"], row["Name"], row["Age"], row["Salary"]))

def clear_entries():
    entry_name.delete(0, tk.END)
    entry_age.delete(0, tk.END)
    entry_salary.delete(0, tk.END)
    entry_search.delete(0, tk.END)

def add_employee():
    global emp_id_counter, employees
    name = entry_name.get().strip()
    age = entry_age.get().strip()
    salary = entry_salary.get().strip()

    if not name or not age or not salary:
        messagebox.showerror("Error", "All fields are required!")
        return
    if not age.isdigit():
        messagebox.showerror("Error", "Age must be a whole number!")
        return
    try:
        salary_val = float(salary)
    except ValueError:
        messagebox.showerror("Error", "Salary must be a number!")
        return

    emp_id = f"Employee{emp_id_counter}"
    new_employee = pd.DataFrame([{
        "ID": emp_id,
        "Name": name,
        "Age": int(age),
        "Salary": salary_val
    }])
    employees = pd.concat([employees, new_employee], ignore_index=True)
    emp_id_counter += 1

    messagebox.showinfo("Success", f"{name} added successfully as {emp_id}!")
    clear_entries()
    refresh_tree()

def search_employee():
    search_val = entry_search.get().strip().lower()
    if not search_val:
        messagebox.showerror("Error", "Please enter Employee ID or Name to search.")
        return

    if search_val.startswith("employee"):
        result = employees[employees["ID"].str.lower() == search_val]
    else:
        result = employees[employees["Name"].str.lower() == search_val]

    for row in tree.get_children():
        tree.delete(row)
    if result.empty:
        messagebox.showinfo("Result", "Employee not found.")
    else:
        for _, row in result.iterrows():
            tree.insert("", tk.END, values=(row["ID"], row["Name"], row["Age"], row["Salary"]))

def delete_employee():
    global employees
    selected_item = tree.selection()
    if not selected_item:
        messagebox.showerror("Error", "Please select an employee to delete.")
        return

    emp_id = tree.item(selected_item[0])["values"][0]
    confirm = messagebox.askyesno("Confirm Deletion", "Are you sure you want to delete {emp_id}?")
    if confirm:
        employees = employees[employees["ID"] != emp_id].reset_index(drop=True)
        refresh_tree()
        messagebox.showinfo("Deleted", "{emp_id} has been deleted.")

def plot_graphs():
    if employees.empty:
        messagebox.showerror("Error", "No employee data to plot.")
        return

    for widget in frame_plot.winfo_children():
        widget.destroy()

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    # Age histogram
    axes[0].hist(employees["Age"], bins=range(min(employees["Age"]), max(employees["Age"]) + 2), color='skyblue', edgecolor='black')
    axes[0].set_title("Age Distribution")
    axes[0].set_xlabel("Age")
    axes[0].set_ylabel("Number of Employees")
    axes[0].grid(axis='y', linestyle='--', alpha=0.7)

    # Salary histogram
    axes[1].hist(employees["Salary"], bins=10, color='lightgreen', edgecolor='black')
    axes[1].set_title("Salary Distribution")
    axes[1].set_xlabel("Salary")
    axes[1].set_ylabel("Number of Employees")
    axes[1].grid(axis='y', linestyle='--', alpha=0.7)

    # Pie chart of salary proportions
    salary_sum = employees["Salary"].sum()
    salaries = employees["Salary"]
    labels = employees["Name"] + " (" + employees["ID"] + ")"
    explode = [0.05] * len(salaries)

    axes[2].pie(salaries, labels=labels, autopct=lambda p: f'{p:.1f} ({p*salary_sum/100:,.0f})', explode=explode,
                shadow=True, startangle=140, textprops={'fontsize': 8})
    axes[2].set_title("Salary Distribution (Pie Chart)")

    plt.tight_layout()

    canvas = FigureCanvasTkAgg(fig, master=frame_plot)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Buttons
ttk.Button(frame_buttons, text="Add Employee", command=add_employee).pack(side=tk.LEFT, padx=5)
ttk.Button(frame_buttons, text="Search Employee", command=search_employee).pack(side=tk.LEFT, padx=5)
ttk.Button(frame_buttons, text="Show All Employees", command=refresh_tree).pack(side=tk.LEFT, padx=5)
ttk.Button(frame_buttons, text="Delete Employee", command=delete_employee).pack(side=tk.LEFT, padx=5)
ttk.Button(frame_buttons, text="Show Graphs", command=plot_graphs).pack(side=tk.LEFT, padx=5)

refresh_tree()
root.mainloop()